In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
import json
import re
from collections import Counter
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool, sum_models
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle

Array = Any

audio + asr

In [ ]:
def load(filename):
    emdeb = np.load(filename)
    return list(emdeb)

In [ ]:
columns = ['clipTarget', 'date', 'asr', 'description']
df = pd.DataFrame(columns=['clipTarget', "audio", 'date', 'asr', 'description'])
for i in range(7):
    csv_filepath = f"/content/drive/MyDrive/ASR/0000{i}.csv"
    audio_filepath = f"/content/drive/MyDrive/ASR/AudioEmbedding/0000{i}.npy"
    temp = pd.read_csv(csv_filepath)
    temp = temp[columns]
    temp["audio"] = load(audio_filepath)
    df = pd.concat([df, temp], ignore_index=True)
    df = df.reset_index(drop=True)
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)

In [ ]:
sorted(set(df['date'].to_list()))

In [ ]:
df[df['date'] == '2022-07-07']

,clipTarget,audio,date
324160,SPORT,"[0.19747002, -0.09017593, -0.15468189, 0.01457...",2022-07-07
324161,TRAVEL,"[0.18064348, -0.08861546, 0.02328031, -0.07119...",2022-07-07
324162,DANCE,"[0.15282989, -0.039978504, -0.12310115, 0.0594...",2022-07-07
324163,ANIMALS,"[0.07372899, -0.08223677, -0.092304066, 0.1175...",2022-07-07
324164,FAMILY,"[0.08605196, -0.03458572, -0.12074478, -0.0027...",2022-07-07
...,...,...,...
360284,VEHICLES,"[0.15986866, 0.0019198233, -0.06104671, 0.0641...",2022-07-07
360285,NATURE,"[0.13996333, -0.13557442, -0.029511726, -0.072...",2022-07-07
360286,SOCIAL,"[0.12375446, -0.06339489, 0.02045422, -0.03073...",2022-07-07
360287,COSPLAY,"[0.19783749, -0.033241857, -0.18903485, 0.0776...",2022-07-07


In [ ]:
len(set(df.iloc[324160:, :]["clipTarget"]))

52

In [ ]:
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
del df

In [ ]:
train_asr = np.load("/content/drive/MyDrive/ASR/fasttext/train_asr.npy")
test_asr = np.load("/content/drive/MyDrive/ASR/fasttext/test_asr.npy")

In [ ]:
index = 0
asr_embed = []
for asr in df_train["asr"].to_list():
    if isinstance(asr, str):
        asr_embed.append(train_asr[index])
        index += 1
    else:
        asr_embed.append(np.zeros(100))

In [ ]:
train_asr.shape, index

((6805, 100), 6805)

In [ ]:
index = 0
asr_embed_test = []
for asr in df_test["asr"].to_list():
    if isinstance(asr, str):
        asr_embed_test.append(test_asr[index])
        index += 1
    else:
        asr_embed_test.append(np.zeros(100))

In [ ]:
test_asr.shape, index

((693, 100), 693)

In [ ]:
asr_embed_test = np.array(asr_embed_test)
asr_embed = np.array(asr_embed)

In [ ]:
X_train = np.concatenate((np.array(df_train["audio"].to_list()), asr_embed), axis=1)
y_train = df_train["clipTarget"].to_list()

In [ ]:
batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(batch)

Learning rate set to 0.211095
0:	learn: 3.6214246	total: 361ms	remaining: 6m 1s
1:	learn: 3.4902857	total: 630ms	remaining: 5m 14s
2:	learn: 3.4042510	total: 868ms	remaining: 4m 48s
3:	learn: 3.3458018	total: 1.15s	remaining: 4m 45s
4:	learn: 3.3002699	total: 1.39s	remaining: 4m 35s
5:	learn: 3.2670761	total: 1.62s	remaining: 4m 28s
6:	learn: 3.2423475	total: 1.85s	remaining: 4m 22s
7:	learn: 3.2236993	total: 2.07s	remaining: 4m 16s
8:	learn: 3.2064044	total: 2.3s	remaining: 4m 13s
9:	learn: 3.1950011	total: 2.5s	remaining: 4m 7s
10:	learn: 3.1821109	total: 2.73s	remaining: 4m 5s
11:	learn: 3.1723576	total: 2.95s	remaining: 4m 2s
12:	learn: 3.1623101	total: 3.19s	remaining: 4m 2s
13:	learn: 3.1509945	total: 3.42s	remaining: 4m
14:	learn: 3.1432493	total: 3.65s	remaining: 3m 59s
15:	learn: 3.1383337	total: 3.87s	remaining: 3m 57s
16:	learn: 3.1297806	total: 4.09s	remaining: 3m 56s
17:	learn: 3.1233029	total: 4.31s	remaining: 3m 55s
18:	learn: 3.1167533	total: 4.54s	remaining: 3m 54s
19:

In [ ]:
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_audio_asr')

In [ ]:
X_test = np.concatenate((np.array(df_test["audio"].to_list()), asr_embed_test), axis=1)
y_test = np.array(df_test["clipTarget"].to_list())

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
f1_score(y_test, y_preds, average='macro')

0.13089045499600338

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.7701186465883593

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.7686988839755429

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.7701186465883593

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.7686988839755429

audio + description

In [ ]:
train_desc = np.load("/content/drive/MyDrive/ASR/fasttext/train_desc.npy")
test_desc = np.load("/content/drive/MyDrive/ASR/fasttext/test_desc.npy")

In [ ]:
index = 0
desc_embed = []
for asr in df_train["description"].to_list():
    if isinstance(asr, str):
        desc_embed.append(train_desc[index])
        index += 1
    else:
        desc_embed.append(np.zeros(100))
        
index = 0
desc_embed_test = []
for asr in df_test["description"].to_list():
    if isinstance(asr, str):
        desc_embed_test.append(test_desc[index])
        index += 1
    else:
        desc_embed_test.append(np.zeros(100))

In [ ]:
desc_embed_test = np.array(desc_embed_test)
desc_embed = np.array(desc_embed)

X_train = np.concatenate((np.array(df_train["audio"].to_list()), desc_embed), axis=1)
y_train = df_train["clipTarget"].to_list()

In [ ]:
batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(batch)

Learning rate set to 0.211095
0:	learn: 3.6251400	total: 355ms	remaining: 5m 54s
1:	learn: 3.4742214	total: 591ms	remaining: 4m 54s
2:	learn: 3.3562716	total: 808ms	remaining: 4m 28s
3:	learn: 3.2809373	total: 1.06s	remaining: 4m 24s
4:	learn: 3.2045003	total: 1.3s	remaining: 4m 18s
5:	learn: 3.1445336	total: 1.54s	remaining: 4m 16s
6:	learn: 3.0904359	total: 1.79s	remaining: 4m 14s
7:	learn: 3.0510292	total: 2.05s	remaining: 4m 13s
8:	learn: 3.0106078	total: 2.3s	remaining: 4m 12s
9:	learn: 2.9818999	total: 2.55s	remaining: 4m 12s
10:	learn: 2.9489716	total: 2.79s	remaining: 4m 11s
11:	learn: 2.9191817	total: 3.03s	remaining: 4m 9s
12:	learn: 2.8948891	total: 3.27s	remaining: 4m 8s
13:	learn: 2.8719743	total: 3.55s	remaining: 4m 10s
14:	learn: 2.8496373	total: 3.79s	remaining: 4m 8s
15:	learn: 2.8289699	total: 4.02s	remaining: 4m 7s
16:	learn: 2.8107034	total: 4.29s	remaining: 4m 7s
17:	learn: 2.7923053	total: 4.52s	remaining: 4m 6s
18:	learn: 2.7776517	total: 4.74s	remaining: 4m 4s
1

In [ ]:
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_audio_desc')

In [ ]:
X_test = np.concatenate((np.array(df_test["audio"].to_list()), desc_embed_test), axis=1)
y_test = np.array(df_test["clipTarget"].to_list())

In [ ]:
y_preds = model.predict(X_test)
f1_score(y_test, y_preds, average='macro')

0.26601638456634197

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.861084192625919

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.8611555104217476

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.861084192625919

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.8611555104217476

audio + description + asr

In [ ]:
X_train = np.concatenate((np.array(df_train["audio"].to_list()), desc_embed, asr_embed), axis=1)
y_train = df_train["clipTarget"].to_list()
batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(batch)

Learning rate set to 0.211095
0:	learn: 3.6251396	total: 427ms	remaining: 7m 6s
1:	learn: 3.4742218	total: 736ms	remaining: 6m 7s
2:	learn: 3.3562716	total: 1.02s	remaining: 5m 38s
3:	learn: 3.2809365	total: 1.36s	remaining: 5m 37s
4:	learn: 3.2045003	total: 1.66s	remaining: 5m 29s
5:	learn: 3.1445332	total: 1.98s	remaining: 5m 27s
6:	learn: 3.0904361	total: 2.29s	remaining: 5m 25s
7:	learn: 3.0510298	total: 2.62s	remaining: 5m 25s
8:	learn: 3.0106074	total: 2.94s	remaining: 5m 23s
9:	learn: 2.9818990	total: 3.27s	remaining: 5m 24s
10:	learn: 2.9489718	total: 3.6s	remaining: 5m 23s
11:	learn: 2.9191813	total: 3.91s	remaining: 5m 21s
12:	learn: 2.8948893	total: 4.23s	remaining: 5m 21s
13:	learn: 2.8719743	total: 4.58s	remaining: 5m 22s
14:	learn: 2.8496375	total: 4.89s	remaining: 5m 20s
15:	learn: 2.8289701	total: 5.19s	remaining: 5m 19s
16:	learn: 2.8107044	total: 5.54s	remaining: 5m 20s
17:	learn: 2.7923045	total: 5.85s	remaining: 5m 18s
18:	learn: 2.7776515	total: 6.12s	remaining: 5m

In [ ]:
# model.save_model('/content/drive/MyDrive/ASR/Catboost/model_audio_desc_asr')

In [ ]:
X_test = np.concatenate((np.array(df_test["audio"].to_list()), desc_embed_test, asr_embed_test), axis=1)
y_test = np.array(df_test["clipTarget"].to_list())

In [ ]:
y_preds = model.predict(X_test)
f1_score(y_test, y_preds, average='macro')

0.2662652134353842

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.8628399540738182

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.8628933225403305

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.8628399540738182

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.8628933225403305

video + asr

In [ ]:
del df_test, df_train

In [ ]:
columns = ['clipTarget', 'date']
df = pd.DataFrame(columns=['clipTarget', "video"])
for i in range(7):
    csv_filepath = f"/content/drive/MyDrive/ASR/0000{i}.csv"
    video_filepath = f"/content/drive/MyDrive/ASR/ContentEmbedding/0000{i}.npy"
    temp = pd.read_csv(csv_filepath)
    temp = temp[columns]
    temp["video"] = load(video_filepath)
    df = pd.concat([df, temp], ignore_index=True)
    df = df.reset_index(drop=True)
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)

In [ ]:
df.sample(3)

,clipTarget,video,date
45323,FOOD,"[0.043309614, 0.0565622, -0.027773185, -0.0123...",2022-04-06
226633,CREATION,"[0.0067632035, -0.015964027, 0.07218018, 0.006...",2022-05-25
369302,HUMOR,"[-0.045005683, -0.06904212, 0.019706415, 0.010...",2022-07-15


In [ ]:
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)


df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
del df

In [ ]:
X_train = np.concatenate((np.array(df_train["video"].to_list()), asr_embed), axis=1)
y_train = df_train["clipTarget"].to_list()

batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(batch)
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_asr')

Learning rate set to 0.211095
0:	learn: 3.3339420	total: 1.09s	remaining: 18m 5s
1:	learn: 3.1029727	total: 1.95s	remaining: 16m 15s
2:	learn: 2.9674929	total: 2.84s	remaining: 15m 44s
3:	learn: 2.8746882	total: 3.84s	remaining: 15m 56s
4:	learn: 2.7947273	total: 4.86s	remaining: 16m 6s
5:	learn: 2.7479250	total: 5.73s	remaining: 15m 49s
6:	learn: 2.6993327	total: 6.59s	remaining: 15m 35s
7:	learn: 2.6529312	total: 7.48s	remaining: 15m 27s
8:	learn: 2.6043379	total: 8.44s	remaining: 15m 28s
9:	learn: 2.5674944	total: 9.4s	remaining: 15m 30s
10:	learn: 2.5369293	total: 10.3s	remaining: 15m 29s
11:	learn: 2.5057342	total: 11.3s	remaining: 15m 30s
12:	learn: 2.4850791	total: 12.2s	remaining: 15m 27s
13:	learn: 2.4554298	total: 13.2s	remaining: 15m 29s
14:	learn: 2.4340185	total: 14s	remaining: 15m 22s
15:	learn: 2.4154108	total: 14.9s	remaining: 15m 18s
16:	learn: 2.3993362	total: 15.8s	remaining: 15m 11s
17:	learn: 2.3761119	total: 16.7s	remaining: 15m 11s
18:	learn: 2.3610272	total: 17.

In [ ]:
X_test = np.concatenate((np.array(df_test["video"].to_list()), asr_embed_test), axis=1)
y_test = np.array(df_test["clipTarget"].to_list())

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
f1_score(y_test, y_preds, average='macro')

0.2874511320353939

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9014046142933748

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.8872455550101408

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9014046142933748

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.8872455550101408

video + description

In [ ]:
X_train = np.concatenate((np.array(df_train["video"].to_list()), desc_embed), axis=1)
y_train = df_train["clipTarget"].to_list()

batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(batch)
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_desc')

Learning rate set to 0.211095
0:	learn: 3.3339416	total: 1.07s	remaining: 17m 48s
1:	learn: 3.1029725	total: 1.92s	remaining: 15m 55s
2:	learn: 2.9674925	total: 2.79s	remaining: 15m 26s
3:	learn: 2.8746888	total: 3.75s	remaining: 15m 34s
4:	learn: 2.7947273	total: 4.73s	remaining: 15m 41s
5:	learn: 2.7350822	total: 5.54s	remaining: 15m 18s
6:	learn: 2.6910355	total: 6.32s	remaining: 14m 55s
7:	learn: 2.6493273	total: 7.19s	remaining: 14m 51s
8:	learn: 2.6008738	total: 8.09s	remaining: 14m 50s
9:	learn: 2.5590950	total: 8.96s	remaining: 14m 47s
10:	learn: 2.5243418	total: 9.82s	remaining: 14m 42s
11:	learn: 2.4880493	total: 10.6s	remaining: 14m 36s
12:	learn: 2.4674702	total: 11.5s	remaining: 14m 34s
13:	learn: 2.4338835	total: 12.3s	remaining: 14m 27s
14:	learn: 2.4072599	total: 13.2s	remaining: 14m 26s
15:	learn: 2.3800130	total: 14.1s	remaining: 14m 29s
16:	learn: 2.3491044	total: 15s	remaining: 14m 26s
17:	learn: 2.3256606	total: 15.8s	remaining: 14m 24s
18:	learn: 2.3030903	total: 

In [ ]:
X_test = np.concatenate((np.array(df_test["video"].to_list()), desc_embed_test), axis=1)
y_test = np.array(df_test["clipTarget"].to_list())

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
f1_score(y_test, y_preds, average='macro')

0.3478577292615149

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9186252315303337

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.9087115606485209

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9186252315303337

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.9087115606485209

video + description + asr

In [ ]:
X_train = np.concatenate((np.array(df_train["video"].to_list()), desc_embed, asr_embed), axis=1)
y_train = df_train["clipTarget"].to_list()

batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(batch)
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_desc_asr')

Learning rate set to 0.211095
0:	learn: 3.3339412	total: 1.18s	remaining: 19m 33s
1:	learn: 3.1029727	total: 2.12s	remaining: 17m 37s
2:	learn: 2.9674925	total: 3.09s	remaining: 17m 6s
3:	learn: 2.8746886	total: 4.17s	remaining: 17m 18s
4:	learn: 2.7947271	total: 5.26s	remaining: 17m 27s
5:	learn: 2.7350822	total: 6.17s	remaining: 17m 2s
6:	learn: 2.6910359	total: 7.03s	remaining: 16m 37s
7:	learn: 2.6493277	total: 8.01s	remaining: 16m 33s
8:	learn: 2.6008734	total: 9.02s	remaining: 16m 33s
9:	learn: 2.5590954	total: 10s	remaining: 16m 30s
10:	learn: 2.5243414	total: 11s	remaining: 16m 24s
11:	learn: 2.4880495	total: 11.9s	remaining: 16m 17s
12:	learn: 2.4674698	total: 12.9s	remaining: 16m 16s
13:	learn: 2.4338837	total: 13.8s	remaining: 16m 9s
14:	learn: 2.4072591	total: 14.8s	remaining: 16m 9s
15:	learn: 2.3800128	total: 15.8s	remaining: 16m 13s
16:	learn: 2.3491042	total: 16.8s	remaining: 16m 9s
17:	learn: 2.3256609	total: 17.7s	remaining: 16m 8s
18:	learn: 2.3030905	total: 18.6s	re

In [ ]:
X_test = np.concatenate((np.array(df_test["video"].to_list()), desc_embed_test, asr_embed_test), axis=1)
y_test = np.array(df_test["clipTarget"].to_list())

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
f1_score(y_test, y_preds, average='macro')

0.34867611474754207

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9172990636308023

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.9073802712161181

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9172990636308023

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.9073802712161181

video + audio + asr

In [ ]:
def load(filename):
    emdeb = np.load(filename)
    return emdeb
    
columns = ['clipTarget', 'date']
df = pd.DataFrame(columns=['clipTarget', "video_audio"])
for i in range(7):
    csv_filepath = f"/content/drive/MyDrive/ASR/0000{i}.csv"
    video_filepath = f"/content/drive/MyDrive/ASR/ContentEmbedding/0000{i}.npy"
    audio_filepath = f"/content/drive/MyDrive/ASR/AudioEmbedding/0000{i}.npy"
    temp = pd.read_csv(csv_filepath)
    temp = temp[columns]
    video = load(video_filepath)
    audio = load(audio_filepath)
    temp["video_audio"] = list(np.concatenate((video, audio), axis=1))
    df = pd.concat([df, temp], ignore_index=True)
    df = df.reset_index(drop=True)
df = df.sort_values(by=['date'], ascending=True)
df = df.reset_index(drop=True)

In [ ]:
df_train = df.iloc[:324160, :]
df_train = df_train.reset_index(drop=True)
X_train = df_train["video_audio"].to_list()
y_train = df_train["clipTarget"].to_list()
del df_train

df_test = df.iloc[324160:, :]
df_test = df_test.reset_index(drop=True)
X_test = df_test["video_audio"].to_list()
y_test = df_test["clipTarget"].to_list()
del df, df_test

In [ ]:
X_train = np.concatenate((np.array(X_train), asr_embed), axis=1)

batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(X=batch)
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_audio_asr')

Learning rate set to 0.211095
0:	learn: 3.3339416	total: 1.19s	remaining: 19m 52s
1:	learn: 3.0870022	total: 2.22s	remaining: 18m 26s
2:	learn: 2.9536701	total: 3.25s	remaining: 17m 59s
3:	learn: 2.8588083	total: 4.29s	remaining: 17m 48s
4:	learn: 2.7756357	total: 5.45s	remaining: 18m 4s
5:	learn: 2.7050538	total: 6.53s	remaining: 18m 1s
6:	learn: 2.6483353	total: 7.58s	remaining: 17m 55s
7:	learn: 2.6065184	total: 8.49s	remaining: 17m 32s
8:	learn: 2.5719915	total: 9.44s	remaining: 17m 19s
9:	learn: 2.5282754	total: 10.5s	remaining: 17m 15s
10:	learn: 2.4897942	total: 11.6s	remaining: 17m 20s
11:	learn: 2.4684498	total: 12.5s	remaining: 17m 12s
12:	learn: 2.4401627	total: 13.6s	remaining: 17m 11s
13:	learn: 2.4125814	total: 14.5s	remaining: 17m 4s
14:	learn: 2.3823116	total: 15.5s	remaining: 17m
15:	learn: 2.3536949	total: 16.6s	remaining: 17m 2s
16:	learn: 2.3292786	total: 17.7s	remaining: 17m 4s
17:	learn: 2.3087655	total: 18.7s	remaining: 17m 2s
18:	learn: 2.2880229	total: 19.6s	re

In [ ]:
X_test = np.concatenate((np.array(X_test), asr_embed_test), axis=1)

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
f1_score(y_test, y_preds, average='macro')

0.31413253190306156

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9156367803492832

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.9030589038869777

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9156367803492832

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.9030589038869777

video + audio + description

In [ ]:
X_train = np.concatenate((np.array(X_train), desc_embed), axis=1)

batch = Pool(X_train, label=y_train)

model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(X=batch)
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_audio_desc')

Learning rate set to 0.211095
0:	learn: 3.3339412	total: 1.19s	remaining: 19m 44s
1:	learn: 3.0870024	total: 2.21s	remaining: 18m 21s
2:	learn: 2.9536697	total: 3.24s	remaining: 17m 57s
3:	learn: 2.8588083	total: 4.29s	remaining: 17m 47s
4:	learn: 2.7756349	total: 5.45s	remaining: 18m 3s
5:	learn: 2.7050538	total: 6.53s	remaining: 18m 1s
6:	learn: 2.6496560	total: 7.52s	remaining: 17m 46s
7:	learn: 2.6103396	total: 8.51s	remaining: 17m 35s
8:	learn: 2.5690186	total: 9.42s	remaining: 17m 17s
9:	learn: 2.5251041	total: 10.4s	remaining: 17m 12s
10:	learn: 2.4913411	total: 11.4s	remaining: 17m 7s
11:	learn: 2.4542699	total: 12.6s	remaining: 17m 13s
12:	learn: 2.4256081	total: 13.5s	remaining: 17m 1s
13:	learn: 2.3995215	total: 14.5s	remaining: 17m 3s
14:	learn: 2.3643820	total: 15.5s	remaining: 16m 58s
15:	learn: 2.3395231	total: 16.7s	remaining: 17m 4s
16:	learn: 2.3091342	total: 17.5s	remaining: 16m 53s
17:	learn: 2.2830167	total: 18.5s	remaining: 16m 50s
18:	learn: 2.2570056	total: 19.4

In [ ]:
X_test = np.concatenate((np.array(X_test), desc_embed_test), axis=1)

In [ ]:
y_preds = model.predict(X_test)
f1_score(y_test, y_preds, average='macro')

0.365972718823688

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9268920541253134

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.9181245282577537

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9268920541253134

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.9181245282577537

video + audio + description + asr

In [ ]:
X_train = np.concatenate((np.array(X_train), desc_embed, asr_embed), axis=1)

batch = Pool(X_train, label=y_train)

In [ ]:
model = CatBoostClassifier(loss_function='MultiClass', task_type="GPU")

model.fit(X=batch)
#model.save_model('/content/drive/MyDrive/ASR/Catboost/model_video_audio_desc_asr')

Learning rate set to 0.211095
0:	learn: 3.3339412	total: 1.24s	remaining: 20m 40s
1:	learn: 3.0870026	total: 2.31s	remaining: 19m 11s
2:	learn: 2.9536699	total: 3.39s	remaining: 18m 45s
3:	learn: 2.8588087	total: 4.48s	remaining: 18m 34s
4:	learn: 2.7756351	total: 5.69s	remaining: 18m 51s
5:	learn: 2.7050536	total: 6.81s	remaining: 18m 48s
6:	learn: 2.6496560	total: 7.84s	remaining: 18m 32s
7:	learn: 2.6103398	total: 8.89s	remaining: 18m 22s
8:	learn: 2.5690186	total: 9.84s	remaining: 18m 3s
9:	learn: 2.5251039	total: 10.9s	remaining: 17m 57s
10:	learn: 2.4913413	total: 11.9s	remaining: 17m 51s
11:	learn: 2.4542701	total: 13.1s	remaining: 17m 57s
12:	learn: 2.4256079	total: 14s	remaining: 17m 42s
13:	learn: 2.3995215	total: 15.1s	remaining: 17m 42s
14:	learn: 2.3643818	total: 16.1s	remaining: 17m 38s
15:	learn: 2.3395233	total: 17.3s	remaining: 17m 43s
16:	learn: 2.3091346	total: 18.2s	remaining: 17m 33s
17:	learn: 2.2830171	total: 19.2s	remaining: 17m 29s
18:	learn: 2.2570060	total: 2

In [ ]:
X_test = np.concatenate((np.array(X_test), desc_embed_test, asr_embed_test), axis=1)

In [ ]:
y_preds = model.predict(X_test)
f1_score(y_test, y_preds, average='macro')

0.36586622331031815

In [ ]:
y_preds_p = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovr")

0.9278001191784913

In [ ]:
roc_auc_score(y_test, y_preds_p, average='macro', multi_class="ovo")

0.9188907729762732

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovr")

0.9278001191784913

In [ ]:
roc_auc_score(y_test, y_preds_p, multi_class="ovo")

0.9188907729762732